In [136]:
import numpy as np
import pandas as pd
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [103]:
df = pd.read_csv('/content/train.csv')

In [104]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [105]:
df.shape

(20800, 5)

In [106]:
del df['id']

In [107]:
df.head()

,title,author,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [108]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [109]:
df.isnull().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [110]:
df.dropna(subset =['text'],inplace=True)

In [111]:
df.isnull().sum()

title      558
author    1918
text         0
label        0
dtype: int64

In [112]:
df.shape

(20761, 4)

In [113]:
df['author'].fillna(value=' ',inplace=True)

In [114]:
df.isnull().sum()

title     558
author      0
text        0
label       0
dtype: int64

In [115]:
del df['title']

In [116]:
print(df.shape)

(20761, 3)


In [117]:
df.head()

,author,text,label
0,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [118]:
df['content'] = df['author'] + " " + df['text']

In [119]:
df.head()

,author,text,label,content
0,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,Daniel J. Flynn Ever get the feeling your life...
2,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss Videos 15 Civilians Killed In ...
4,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Print \nAn Iranian woman has be...


In [121]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    processed_text = ' '.join(words)
    return processed_text


df['content'] = df['content'].apply(preprocess_text)


print(df['content'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0        darrel lucu hous dem aid didnt even see comey ...
1        daniel j flynn ever get feel life circl rounda...
2        consortiumnewscom truth might get fire octob 2...
3        jessica purkiss video 15 civilian kill singl u...
4        howard portnoy print iranian woman sentenc six...
                               ...                        
20795    jerom hudson rapper unload black celebr met do...
20796    benjamin hoffman green bay packer lost washing...
20797    michael j de la merc rachel abram maci today g...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson david swanson author activist jo...
Name: content, Length: 20761, dtype: object


In [122]:
df.head()

,author,text,label,content
0,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hous dem aid didnt even see comey ...
1,Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn ever get feel life circl rounda...
2,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnewscom truth might get fire octob 2...
3,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss video 15 civilian kill singl u...
4,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy print iranian woman sentenc six...


In [123]:
X = df.iloc[:,-1]
X

0        darrel lucu hous dem aid didnt even see comey ...
1        daniel j flynn ever get feel life circl rounda...
2        consortiumnewscom truth might get fire octob 2...
3        jessica purkiss video 15 civilian kill singl u...
4        howard portnoy print iranian woman sentenc six...
                               ...                        
20795    jerom hudson rapper unload black celebr met do...
20796    benjamin hoffman green bay packer lost washing...
20797    michael j de la merc rachel abram maci today g...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson david swanson author activist jo...
Name: content, Length: 20761, dtype: object

In [125]:
y = df.iloc[:,2]
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20761, dtype: int64

In [127]:
vec = TfidfVectorizer()

In [128]:
X = vec.fit_transform(X)

In [131]:
print(X)

  (0, 143351)	0.05042567133431003
  (0, 83360)	0.04783222595830813
  (0, 24628)	0.021803608431334387
  (0, 29588)	0.024975455489409264
  (0, 47163)	0.019116344360645264
  (0, 31655)	0.04007769900200579
  (0, 35566)	0.05090755300218484
  (0, 50023)	0.01265404026559739
  (0, 89610)	0.04196507780183673
  (0, 81494)	0.02740635321538867
  (0, 129238)	0.04625271255818686
  (0, 7975)	0.023614520728112542
  (0, 125400)	0.02333029261890401
  (0, 65397)	0.018071280864480562
  (0, 118792)	0.027355973783887325
  (0, 114450)	0.028102808804626393
  (0, 122545)	0.01741127783289736
  (0, 37728)	0.024538963065789923
  (0, 76857)	0.020156044102836695
  (0, 133627)	0.039050915825816525
  (0, 28677)	0.048513227093376715
  (0, 27649)	0.03941152102279975
  (0, 90894)	0.05090755300218484
  (0, 124462)	0.02641076302655544
  (0, 30512)	0.021287757728390887
  :	:
  (20760, 80245)	0.012021712650103576
  (20760, 55433)	0.013516584131082874
  (20760, 108461)	0.01952160843210621
  (20760, 55507)	0.02082578150061475

In [133]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.2 , stratify = y , random_state = 2)

In [134]:
model = LogisticRegression()

In [135]:
model.fit(X_train , y_train)

LogisticRegression()

In [137]:
y_pred = model.predict(X_test)

In [138]:
score = accuracy_score(y_pred , y_test)

In [140]:
print("the accuracy of the model is " , score )

the accuracy of the model is  0.954972309174091


In [152]:
input = "House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It"
list = []
text = input.lower()
text = re.sub(r'[^\w\s]', '', text)
words = word_tokenize(text)
stop_words = set(stopwords.words('english'))
words = [word for word in words if word not in stop_words]
stemmer = PorterStemmer()
words = [stemmer.stem(word) for word in words]
processed_text = ' '.join(words)
list.append(processed_text)
input = vec.transform(list)

In [153]:
print(input)

  (0, 132682)	0.2883681858596941
  (0, 115743)	0.15810924468262508
  (0, 76590)	0.27355140942234685
  (0, 68663)	0.35598192550016644
  (0, 61791)	0.1725600292244693
  (0, 46061)	0.13451874612480855
  (0, 38463)	0.21903707448215615
  (0, 36930)	0.42252817482262905
  (0, 30776)	0.31908196283252976
  (0, 27361)	0.5035114966036508
  (0, 9638)	0.24612003122750487


In [154]:
model.predict(input)

array([1])